<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Todo: é preciso fazer a varredura de todos os diretórios src contendo .java

# Checa ambiente

In [1]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [2]:
!python3 --version

Python 3.10.12


In [3]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (332 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120831 files and directories currently install

# Funcões

In [14]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd kafka && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [5]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

#core/src/main/java
def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['KAFKA'] = tag + 'core/src/main/java'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $KAFKA -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [6]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

'''
Example usage:
url1 = "https://www.example.com"
url2 = "https://www.invalid-url.com"
print(is_valid_url(url1))  # Output: True
print(is_valid_url(url2))  # Output: False
'''

'\nExample usage:\nurl1 = "https://www.example.com"\nurl2 = "https://www.invalid-url.com"\nprint(is_valid_url(url1))  # Output: True\nprint(is_valid_url(url2))  # Output: False\n'

In [7]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [8]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [9]:
!git clone https://github.com/apache/kafka.git

Cloning into 'kafka'...
remote: Enumerating objects: 345347, done.
remote: Total 345347 (delta 0), reused 0 (delta 0), pack-reused 345347
Receiving objects: 100% (345347/345347), 160.70 MiB | 18.84 MiB/s, done.
Resolving deltas: 100% (166539/166539), done.


In [ ]:
!tree kafka

### Pega a lista de tags

In [11]:
lista_tags_kafka = !cd kafka && git tag

In [12]:
pattern_to_extract = r'3.'
get_elements_with_pattern(lista_tags_kafka, pattern_to_extract)

['0.11.0.3-rc0',
 '2.3.0',
 '2.3.0-rc1',
 '2.3.0-rc2',
 '2.3.0-rc3',
 '2.3.1',
 '2.3.1-rc0',
 '2.3.1-rc1',
 '2.3.1-rc2',
 '2.6.3-rc0',
 '3.0.0',
 '3.0.0-rc0',
 '3.0.0-rc1',
 '3.0.0-rc2',
 '3.0.1',
 '3.0.1-rc0',
 '3.0.2',
 '3.0.2-rc0',
 '3.1.0',
 '3.1.0-rc0',
 '3.1.0-rc1',
 '3.1.1',
 '3.1.1-rc0',
 '3.1.1-rc1',
 '3.1.2',
 '3.1.2-rc0',
 '3.2.0',
 '3.2.0-rc0',
 '3.2.0-rc1',
 '3.2.1',
 '3.2.1-rc2',
 '3.2.1-rc3',
 '3.2.2',
 '3.2.2-rc0',
 '3.2.3',
 '3.2.3-rc0',
 '3.3.0',
 '3.3.0-rc1',
 '3.3.0-rc2',
 '3.3.1',
 '3.3.1-rc0',
 '3.3.2',
 '3.3.2-rc0',
 '3.3.2-rc1',
 '3.4.0',
 '3.4.0-rc0',
 '3.4.0-rc1',
 '3.4.0-rc2',
 '3.4.1',
 '3.4.1-rc0',
 '3.4.1-rc1',
 '3.4.1-rc2',
 '3.4.1-rc3',
 '3.5.0',
 '3.5.0-rc0',
 '3.5.0-rc1',
 '3.5.1',
 '3.5.1-rc0',
 '3.5.1-rc1']

# Análise da tag kafka_3_0_1

In [15]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_3_0_1 = get_commit_tag(tag='3.0.1')
cria_pasta_n_commit(repositorio='kafka', pasta_n_commit='commit_tag_3_0_1')
checkout_n_commit('commit_tag_3_0_1', commit_tag_3_0_1[0])

Checkout do repositorio no commit 8e30984f43e64d8b9c2715c431a8906c98d26b86 realizado com sucesso!


In [17]:
ls -liath commit_tag_3_0_1/core/src/main/java

total 12K
2237521 drwxr-xr-x 3 root root 4.0K Aug 27 13:57 ./
2238162 drwxr-xr-x 5 root root 4.0K Aug 27 13:57 ../
2237522 drwxr-xr-x 3 root root 4.0K Aug 27 13:57 kafka/


In [18]:
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/core/src/main/java -o commit_tag_3_0_1

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [19]:
!ls -liat commit_tag_3_0_1/designCodeSmells.csv

2235388 -rw-r--r-- 1 root root 109 Aug 27 13:58 commit_tag_3_0_1/designCodeSmells.csv


In [20]:
df_design_smells = pd.read_csv('commit_tag_3_0_1/designCodeSmells.csv')

In [21]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,java,kafka.metrics,KafkaYammerMetrics,Deficient Encapsulation


In [22]:
df_design_smells['Type Name'].unique()

array(['KafkaYammerMetrics'], dtype=object)

In [23]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
df_temp_kafka = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/kafka-treemap/main/results/kafka_java_20_critical.csv')
df_temp_kafka

,Unnamed: 0,File,Frequency,lines_modified,fcxflc
0,452,StreamThread.java,323,14137,4566251
1,168,KafkaConsumer.java,287,10142,2910754
2,403,KStream.java,121,18622,2253262
3,1372,KafkaAdminClient.java,191,10647,2033577
4,451,StreamTask.java,212,7250,1537000
5,233,Fetcher.java,195,7834,1527630
6,760,KafkaStreams.java,201,7232,1453632
7,1555,TaskManager.java,158,7917,1250886
8,419,KStreamImpl.java,134,8558,1146772
9,35,KafkaProducer.java,218,4765,1038770


In [24]:
lista_classes_criticas = df_temp_kafka.File.tolist()
lista_classes_criticas

['StreamThread.java',
 'KafkaConsumer.java',
 'KStream.java',
 'KafkaAdminClient.java',
 'StreamTask.java',
 'Fetcher.java',
 'KafkaStreams.java',
 'TaskManager.java',
 'KStreamImpl.java',
 'KafkaProducer.java',
 'StreamsConfig.java',
 'ConsumerCoordinator.java',
 'Sender.java',
 'AbstractCoordinator.java',
 'KTableImpl.java',
 'KTable.java',
 'StreamsPartitionAssignor.java',
 'NetworkClient.java',
 'InternalTopologyBuilder.java',
 'Protocol.java']

In [25]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [26]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell


In [27]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell


In [29]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('commit_tag_3_0_1/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('commit_tag_3_0_1/analise_designite.csv', index=False)

# Análise da tag 3_5_1

In [30]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_3_5_1 = get_commit_tag(tag='3.5.1')
cria_pasta_n_commit(repositorio='kafka', pasta_n_commit='commit_tag_3_5_1')
checkout_n_commit('commit_tag_3_5_1', commit_tag_3_5_1[0])

Checkout do repositorio no commit 2c6fb6c54472e90ae17439e62540ef3cb0426fe3 realizado com sucesso!


In [31]:
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/core/src/main/java -o commit_tag_3_5_1

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [32]:
df_design_smells_final = pd.read_csv('commit_tag_3_5_1/designCodeSmells.csv')

In [37]:
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,java,kafka.server.builders,KafkaApisBuilder,Unutilized Abstraction
1,java,kafka.server.builders,KafkaApisBuilder,Insufficient Modularization
2,java,kafka.server.builders,ReplicaManagerBuilder,Unutilized Abstraction
3,java,kafka.server.builders,ReplicaManagerBuilder,Insufficient Modularization
4,java,kafka.server.builders,LogManagerBuilder,Unutilized Abstraction
5,java,kafka.server,ReplicaAlterLogDirsTierStateMachine,Unutilized Abstraction
6,java,kafka.server,TierStateMachine,Unutilized Abstraction
7,java,kafka.server,ReplicaFetcherTierStateMachine,Unutilized Abstraction
8,java,kafka.log.remote,RemoteLogManager,Insufficient Modularization
9,java,kafka.log.remote,CancellableRunnable,Unutilized Abstraction


In [34]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [41]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [42]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell


In [43]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('commit_tag_3_5_1/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('commit_tag_3_5_1/analise_designite.csv', index=False)